In [1]:
from mlrun import get_or_create_ctx, mlconf
from mlrun.artifacts import TableArtifact

In [2]:
# nuclio: ignore
mlrunctx = get_or_create_ctx('mlrun')

[mlrun] 2020-03-14 20:28:53,206 logging run results to: http://10.196.67.76:8080


In [3]:
# Import needed functions/classes
from netapp_ontap import config as netappConfig
from netapp_ontap.host_connection import HostConnection as NetAppHostConnection
from netapp_ontap.resources import Volume, Snapshot
from netapp_ontap import config, utils

from datetime import datetime
import json
import sys, subprocess
import os


In [7]:
def netappClone(
    context,
    ontapClusterMgmtHostname,
    ontapClusterAdminUsername,
    ontapClusterAdminPassword,
    sourceVolumeName,
    verifySSLCert
):
    
    is_simulated = context.get_param('NETAPP_SIM',False)
    NETAPP_MOUNT_PATH = context.get_param('NETAPP_MOUNT_PATH','/User/nfs')
    print("SIMULATED",is_simulated,ontapClusterMgmtHostname,context)
    if is_simulated:
        clone_path = os.path.join(NETAPP_MOUNT_PATH,"clone")
        context.log_result('cloneVolumeDetails',clone_path)
        context.logger.info('Clone volume name: %s'%clone_path)
        return clone_path
    print("====>",ontapClusterMgmtHostname)
    # Configure connection to ONTAP cluster/instance
    netappConfig.CONNECTION = NetAppHostConnection(
        host = ontapClusterMgmtHostname,
        username = ontapClusterAdminUsername,
        password = ontapClusterAdminPassword,
        verify = verifySSLCert
    )
   
    
    #print('\nSource pv name: ', sourcePvName)
    context.logger.info('Source ONTAP volume name: %s'%sourceVolumeName)
 
    # Create clone
    
    sourceVolume = Volume.find(name = sourceVolumeName)
    timestamp = datetime.today().strftime("%Y%m%d_%H%M%S")
    cloneVolumeName = 'kfp_clone_%s' % timestamp
    cloneVolume = Volume.from_dict({
        'name': cloneVolumeName,
        'svm': sourceVolume.to_dict()['svm'],
        'clone': {
            'is_flexclone':'true',
            'parent_volume': sourceVolume.to_dict()
        },
        'nas': {
            'path': '/%s' % cloneVolumeName
        }
    })
    response = cloneVolume.post()
    print("\nAPI Response:")
    print(response.http_response.text)
 
    # Retrieve clone volume details
    cloneVolume.get()
 
    # Convert clone volume details to JSON string
    cloneVolumeDetails = cloneVolume.to_dict()
    print("\nClone Volume Details:")
    print(json.dumps(cloneVolumeDetails, indent=2))
 
    clone_path = os.path.join(NETAPP_MOUNT_PATH,cloneVolumeDetails['name'])
    context.log_result('cloneVolumeDetails',clone_path)
    # Return name of new clone volume
    return clone_path

In [ ]:
def get_latest_dataset(context,clone_path):
    features_table=os.path.join(clone_path,context.get_param('FEATURES_TABLE', 'netops_features'))
    # Get parquet files
    mpath = [os.path.join(features_table, file) for file in os.listdir(features_table)]
    
    # Get latest filename
    latest = max(mpath, key=os.path.getmtime)
    context.log_result('training_parquet_file',latest)
    context.logger.debug(f'Reading data from: {latest}')
    
    return ""

In [13]:
def handler(context,
    ontapClusterMgmtHostname  = 'localhost',
    ontapClusterAdminUsername = 'admin',
    ontapClusterAdminPassword = 'NetApp!23',
    sourceVolumeName = 'trident_default_tenant_basic_08204',
    verifySSLCert: bool = False
):
    # Clone NetApp Volume
    clone_path  = netappClone(context,ontapClusterMgmtHostname,ontapClusterAdminUsername,ontapClusterAdminPassword,sourceVolumeName,verifySSLCert)
    
    get_latest_dataset(context,clone_path)
    # Get latest data set for training
    
    
    

In [ ]:
# nuclio: ignore
import logging
logging.basicConfig(level=logging.DEBUG)
output = netappClone(mlrunctx,ontapClusterMgmtHostname = '3.135.218.180',
ontapClusterAdminUsername = 'marcelol@iguazio.com',
ontapClusterAdminPassword = 'd@ta1sg00d',
sourceVolumeName = 'kubernetes_trident_pvc_551720fa_3758_461b_ac2c_fb688b676e4c')
output